In [1]:
import os
import sys
os.environ["PYSPARK_PYTHON"] = "/opt/cloudera/parcels/Anaconda/bin/python"
os.environ["JAVA_HOME"] = "/usr/java/jdk1.8.0_161/jre"
os.environ["SPARK_HOME"] = "/opt/cloudera/parcels/SPARK2-2.3.0.cloudera2-1.cdh5.13.3.p0.316101/lib/spark2/"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.6-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('project').master("local").enableHiveSupport().getOrCreate()

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1662629675872_0004,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
#Data Ingestion
#Reading data from HDFS and listing
df = spark.read.format("csv").load("hdfs:///user/hadoop/sqoop_import/")
df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-------+---+------+---+--------+---+---------------+------------------+--------------------+----+----+------+------+----+--------------------+----+----------+----+----+------+------+-------+--------------+-------+----+----+----+----+-----+----+----+-------+--------------------+
| _c0|    _c1|_c2|   _c3|_c4|     _c5|_c6|            _c7|               _c8|                 _c9|_c10|_c11|  _c12|  _c13|_c14|                _c15|_c16|      _c17|_c18|_c19|  _c20|  _c21|   _c22|          _c23|   _c24|_c25|_c26|_c27|_c28| _c29|_c30|_c31|   _c32|                _c33|
+----+-------+---+------+---+--------+---+---------------+------------------+--------------------+----+----+------+------+----+--------------------+----+----------+----+----+------+------+-------+--------------+-------+----+----+----+----+-----+----+----+-------+--------------------+
|2017|January|  1|Sunday|  0|  Active|  1|            NCR|        NÃƒÂ¦stved|         Farimagsvej|   8|4700|55.233|11.763| DKK|          MasterCa

In [3]:
#create input schema using StructType

from pyspark.sql.types import*

shm = StructType([
    StructField("year",IntegerType(),False),
    StructField("month", StringType(),False),
    StructField("day",IntegerType(),False),
    StructField("weekday",StringType(),False),
    StructField("hour",IntegerType(),False),
    StructField("atm_status",StringType(),False),
    StructField("atm_id",StringType(),False),
    StructField("atm_manufacturer",StringType(),False),
    StructField("atm_location",StringType(),False),
    StructField("atm_streetname",StringType(),False),
    StructField("atm_street_number",IntegerType(),False),
    StructField("atm_zip_code",IntegerType(),False),
    StructField("atm_lat",DecimalType(),False),
    StructField("atm_lon",DecimalType(),False),
    StructField("currency",StringType(),False),
    StructField("card_type",StringType(),False),
    StructField("transaction_amount",IntegerType(),False),
    StructField("service",StringType(),False),
    StructField("message_code",StringType(),True),
    StructField("message_text",StringType(),True),
    StructField("weather_lat",DecimalType(),False),
    StructField("weather_lon",DecimalType(),False),
    StructField("weather_city_id",IntegerType(),False),
    StructField("weather_city_name",StringType(),False),
    StructField("temp",DecimalType(),False),
    StructField("pressure",IntegerType(),False),
    StructField("humidity",IntegerType(),False),
    StructField("wind_speed",IntegerType(),False),
    StructField("wind_deg",IntegerType(),False),
    StructField("rain_3h",DecimalType(),True),
    StructField("clouds_all",IntegerType(),False),
    StructField("weather_id",IntegerType(),False),
    StructField("weather_main",StringType(),False),
    StructField("weather_description",StringType(),False)
])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
# read the data using the input schema created 
# verifying the data using count function
df1 = spark.read.option('delimeter','|').csv('hdfs:///user/hadoop/sqoop_import/',schema=shm)
df1.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2468572

In [5]:
#create a dataframe for Date Dimension according to Target Dimension model

import pyspark
from pyspark.sql.types import*
from pyspark.sql.functions import*

df_date = df1.select(col("year"),col("month"),col("day"),col("hour"),col("weekday")).distinct()
df_date.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

8685

In [6]:
df_date.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- year: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- day: integer (nullable = true)
 |-- hour: integer (nullable = true)
 |-- weekday: string (nullable = true)

In [7]:
#clean and transform the data for Date dimension - add primary key (date_id)
from pyspark.sql.functions import monotonically_increasing_id

dim_date = df_date.withColumn("date_id", monotonically_increasing_id())
dim_date.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+--------+---+----+---------+-------+
|year|   month|day|hour|  weekday|date_id|
+----+--------+---+----+---------+-------+
|2017| January|  5|  21| Thursday|      0|
|2017| January| 22|  15|   Sunday|      1|
|2017|   April|  7|   9|   Friday|      2|
|2017| January| 23|  18|   Monday|      3|
|2017|   March| 17|   1|   Friday|      4|
|2017|   March| 27|  13|   Monday|      5|
|2017|   March| 29|   7|Wednesday|      6|
|2017|   March|  4|   8| Saturday|      7|
|2017|   March| 12|  12|   Sunday|      8|
|2017| January|  4|   4|Wednesday|      9|
|2017|February|  8|  13|Wednesday|     10|
|2017|February| 18|  21| Saturday|     11|
|2017|February| 28|  21|  Tuesday|     12|
|2017| January| 24|  14|  Tuesday|     13|
|2017|February|  6|  15|   Monday|     14|
|2017|   March| 20|   9|   Monday|     15|
|2017| January| 27|   2|   Friday|     16|
|2017|   March|  4|   4| Saturday|     17|
|2017| January| 14|   5| Saturday|     18|
|2017| January| 16|  19|   Monday|     19|
+----+-----

In [8]:
#Check count for the Date Dimension
#Count of Records - 8685
dim_date.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

8685

In [9]:
#create a dataframe for Card Type Dimension according to Target Dimension model
#clean and transform the data for Card Type dimension

#card_type dimension
#Check count for the Card Type Dimension
#Count of Records - 12

dim_card_type = df1.select(col("card_type")).distinct()
dim_card_type.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

12

In [10]:
#create a dataframe for Location Dimension according to Target Dimension model
#clean and transform the data for Location dimension

#Check count for the Location Dimension
#Count of Records - 109

dim_location = df1.select(col("atm_location"),col("atm_lat"),col("atm_lon"),col("atm_streetname"),col("atm_street_number"),col("atm_zip_code")).distinct()
dim_location.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

109

In [11]:
#create a dataframe for ATM Dimension according to Target Dimension model
#clean and transform the data for ATM dimension

#Check count for the ATM Dimension
#Count of Records - 113

dim_atm = df1.select(col("atm_id"),col("atm_manufacturer"),col("atm_location")).distinct()
dim_atm.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

113

In [12]:
#schema for fact table

schema_fact = StructType([
    StructField("id",IntegerType(),False),
    StructField("year",IntegerType(),False),
    StructField("month", StringType(),False),
    StructField("day",IntegerType(),False),
    StructField("hour",StringType(),False),
    StructField("weekday",IntegerType(),False),
    StructField("card_type",StringType(),False),
    StructField("atm_location",StringType(),False),
    StructField("atm_lat",DecimalType(),False),
    StructField("atm_lon",DecimalType(),False),
    StructField("atm_streetname",StringType(),False),
    StructField("atm_street_number",IntegerType(),False),
    StructField("atm_zip_code",IntegerType(),False),
    StructField("atm_status",StringType(),False),
    StructField("atm_id",StringType(),False),
    StructField("atm_manufacturer",StringType(),False)  
])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
#primary key for fact table

dim_fact = df1.withColumn("id", monotonically_increasing_id())
dim_fact.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- year: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- day: integer (nullable = true)
 |-- weekday: string (nullable = true)
 |-- hour: integer (nullable = true)
 |-- atm_status: string (nullable = true)
 |-- atm_id: string (nullable = true)
 |-- atm_manufacturer: string (nullable = true)
 |-- atm_location: string (nullable = true)
 |-- atm_streetname: string (nullable = true)
 |-- atm_street_number: integer (nullable = true)
 |-- atm_zip_code: integer (nullable = true)
 |-- atm_lat: decimal(10,0) (nullable = true)
 |-- atm_lon: decimal(10,0) (nullable = true)
 |-- currency: string (nullable = true)
 |-- card_type: string (nullable = true)
 |-- transaction_amount: integer (nullable = true)
 |-- service: string (nullable = true)
 |-- message_code: string (nullable = true)
 |-- message_text: string (nullable = true)
 |-- weather_lat: decimal(10,0) (nullable = true)
 |-- weather_lon: decimal(10,0) (nullable = true)
 |-- weather_city_id: integer (nullable = tru

In [14]:
#create fact table

#Check count for the all the Stages in the creation of Transaction Fact table
#Count of Records in each stage - 2468572

df_fact = dim_fact.select(col("id"),col("year"),col("month"),col("day"),col("hour"),col("weekday"),col("card_type"),col("atm_location"),col("atm_lat"),col("atm_lon"),col("atm_streetname"),col("atm_street_number"),col("atm_zip_code"),col("atm_status"),col("atm_id"),col("atm_manufacturer")).distinct()
df_fact.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2468572

In [17]:
#load date dimension to S3

dim_date.coalesce(1).write.csv("s3://myfactanddimensiontables/dim_date/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
#load card_type dimension to S3

dim_card_type.coalesce(1).write.csv("s3://myfactanddimensiontables/dim_card_type/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
#load location dimension to S3

dim_location.coalesce(1).write.csv("s3://myfactanddimensiontables/dim_location/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
#load atm dimension to S3

dim_atm.coalesce(1).write.csv("s3://myfactanddimensiontables/dim_atm/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
#load fact table to S3

df_fact.coalesce(1).write.csv("s3://myfactanddimensiontables/fact_table/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…